In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!cp /content/gdrive/My\ Drive/data/*.zip .
!unzip /content/sudoku.zip
!mkdir /content/test
!unzip /content/sudoku_test.zip -d /content/test
!mv /content/test/sudoku.csv /content/sudoku_test.csv

In [ ]:
!git clone https://github.com/cloughurd/drl-sudoku.git
!mv drl-sudoku/data/* .

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import librosa
import os
import gc
import wave
import struct
import math
import contextlib
from mido import MidiFile
from mido.messages.messages import Message
import mido
from typing import List
from tqdm import tqdm
import random
from torch.utils.data.sampler import SubsetRandomSampler


from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)